In [1]:
import sys


import torch
torch.set_printoptions(linewidth=200, threshold=100000)
import math
from mlstm_parallel._torch_fwbw_legacy import mlstm_fw, mlstm_fw_legacy
from mlstm_parallel._torch_fwbw_legacy import mlstm_bw, mlstm_bw_legacy

In [2]:
S = 6 #32 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float64
DEVICE = torch.device("cuda:0")
BLOCK_Q = 16
BLOCK_KV = 16

In [3]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# qs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# # vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 100.
# # vs = torch.zeros((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# vs[:,:,1,0] = 7.
# qs[:,:,1,0] = 1.

# vs[:,:,1,16] = 8.
# qs[:,:,1,16] = 1.
# random: 
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S) / 10.
# igs = torch.zeros((B, NH, S), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S), device=DEVICE, dtype=DTYPE) #/ 10.
# fgs = torch.ones((B, NH, S), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S), device=DEVICE, dtype=DTYPE)

dH = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

vM = torch.randn((B, NH, S), device=DEVICE, dtype=DTYPE)
vN = torch.randn((B, NH, S), device=DEVICE, dtype=DTYPE)

In [4]:
h_r, m_r, n_r, lfg_r = mlstm_fw(qs, ks, vs, igs, fgs)

In [5]:
h_l, m_l, n_l, lfg_l = mlstm_fw_legacy(qs, ks, vs, igs.unsqueeze(-1), fgs.unsqueeze(-1))

In [6]:
h_r - h_l

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.7756e-17,  2.7756e-17,  5.5511e-17, -5.5511e-17, -1.1102e-16,  5.5511e-17, -1.1102e-16, -2.7756e-17],
          [ 2.2204e-16, -1.1102e-16, -5.5511e-17,  2.0817e-17,  2.2204e-16, -1.1102e-16, -2.7756e-17, -2.2204e-16],
          [ 0.0000e+00,  2.7756e-17,  1.7347e-18,  0.0000e+00,  0.0000e+00,  2.7756e-17,  0.0000e+00,  0.0000e+00],
          [-1.1102e-16,  2.2204e-16, -2.7756e-17, -2.2204e-16, -8.3267e-17,  3.4694e-17, -2.2204e-16,  1.1102e-16]]]], device='cuda:0', dtype=torch.float64)

In [7]:
m_r - m_l.squeeze(-1)

tensor([[[0., 0., 0., 0., 0., 0.]]], device='cuda:0', dtype=torch.float64)

In [8]:
n_r - n_l.squeeze(-1)

tensor([[[-2.2204e-16,  2.2204e-16,  0.0000e+00,  0.0000e+00,  0.0000e+00, -5.5511e-17]]], device='cuda:0', dtype=torch.float64)

In [9]:
lfg_r - lfg_l

tensor([[[[0., nan, nan, nan, nan, nan],
          [0., 0., nan, nan, nan, nan],
          [0., 0., 0., nan, nan, nan],
          [0., 0., 0., 0., nan, nan],
          [0., 0., 0., 0., 0., nan],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0', dtype=torch.float64)

In [10]:
lfg_r

tensor([[[[ 0.0000,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.1650,  0.0000,    -inf,    -inf,    -inf,    -inf],
          [-3.2819, -2.1168,  0.0000,    -inf,    -inf,    -inf],
          [-4.6425, -3.4775, -1.3607,  0.0000,    -inf,    -inf],
          [-6.5368, -5.3718, -3.2550, -1.8943,  0.0000,    -inf],
          [-7.1744, -6.0094, -3.8925, -2.5319, -0.6376,  0.0000]]]], device='cuda:0', dtype=torch.float64)

In [11]:
lfg_l

tensor([[[[ 0.0000,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.1650,  0.0000,    -inf,    -inf,    -inf,    -inf],
          [-3.2819, -2.1168,  0.0000,    -inf,    -inf,    -inf],
          [-4.6425, -3.4775, -1.3607,  0.0000,    -inf,    -inf],
          [-6.5368, -5.3718, -3.2550, -1.8943,  0.0000,    -inf],
          [-7.1744, -6.0094, -3.8925, -2.5319, -0.6376,  0.0000]]]], device='cuda:0', dtype=torch.float64)

In [12]:
dq_r, dk_r, dv_r, di_r, df_r, dc_r, dd_r, d_r, c_r = mlstm_bw(dH, qs, ks, vs, igs, fgs, vM, vN)

In [13]:
dq_l, dk_l, dv_l, di_l, df_l, dc_l, dd_l, d_l, c_l = mlstm_bw_legacy(dH, qs, ks, vs, igs.unsqueeze(-1), fgs.unsqueeze(-1), vM.unsqueeze(-1), vN.unsqueeze(-1))

In [14]:
dc_r - dc_l

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0', dtype=torch.float64)

In [15]:
dd_r - dd_l

tensor([[[[-1.7764e-15,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-4.4409e-16,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 5.5511e-17, -2.2204e-16,  2.2204e-16,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 2.7756e-17,  2.7756e-17,  4.4409e-16,  8.8818e-16,  0.0000e+00,  0.0000e+00],
          [ 4.3368e-19, -1.3878e-17, -1.7347e-18,  1.6653e-16,  0.0000e+00,  0.0000e+00],
          [ 8.6736e-19,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.4211e-14, -7.1054e-15]]]], device='cuda:0', dtype=torch.float64)

In [16]:
d_r - d_l

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0', dtype=torch.float64)

In [17]:
c_r - c_l

tensor([[[[-3.3307e-16,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 2.2204e-16, -3.5527e-15,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.3878e-17, -5.5511e-17,  2.2204e-16,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 6.9389e-18,  2.2204e-16, -2.2204e-16, -8.8818e-16,  0.0000e+00,  0.0000e+00],
          [ 5.4210e-20, -1.3010e-18, -6.9389e-18,  4.1633e-17,  0.0000e+00,  0.0000e+00],
          [ 2.7105e-20,  0.0000e+00,  0.0000e+00,  4.3368e-19, -1.1102e-16,  2.7756e-17]]]], device='cuda:0', dtype=torch.float64)

In [18]:
dq_r - dq_l

tensor([[[[ 0.0000e+00,  0.0000e+00,  4.4409e-16,  0.0000e+00,  0.0000e+00, -8.8818e-16, -2.2204e-16,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  1.7764e-15,  0.0000e+00, -7.1054e-15,  0.0000e+00, -1.7764e-15,  0.0000e+00],
          [-5.5511e-17, -5.5511e-17, -6.9389e-17,  0.0000e+00,  5.5511e-17, -1.1102e-16,  0.0000e+00,  6.9389e-18],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  4.1633e-17,  4.1633e-17, -2.2204e-16,  4.4409e-16,  0.0000e+00],
          [ 0.0000e+00,  2.7756e-17,  0.0000e+00,  2.2204e-16,  1.1102e-16,  0.0000e+00,  2.2204e-16, -4.4409e-16],
          [-1.4211e-14,  8.8818e-16,  2.6645e-15, -3.5527e-15,  0.0000e+00, -7.1054e-15,  7.1054e-15,  0.0000e+00]]]], device='cuda:0', dtype=torch.float64)

In [19]:
dk_r - dk_l

tensor([[[[ 0.0000e+00,  1.1102e-16,  2.2204e-16,  1.7764e-15,  0.0000e+00,  2.2204e-16,  4.4409e-16,  1.1102e-16],
          [ 8.8818e-16, -1.7764e-15,  3.5527e-15,  3.5527e-15,  1.4211e-14,  0.0000e+00, -3.5527e-15,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.1102e-16,  1.3878e-17, -2.7756e-17,  5.5511e-17],
          [ 0.0000e+00,  1.1102e-16,  2.2204e-16,  4.4409e-16,  0.0000e+00,  1.1102e-16,  0.0000e+00, -5.5511e-17],
          [ 0.0000e+00, -1.7764e-15,  0.0000e+00,  0.0000e+00, -8.8818e-16,  2.2204e-16, -1.7764e-15, -8.8818e-16],
          [ 0.0000e+00,  7.1054e-15,  1.4211e-14,  0.0000e+00,  3.5527e-15, -8.8818e-16,  0.0000e+00,  0.0000e+00]]]], device='cuda:0', dtype=torch.float64)

In [20]:
dv_r - dv_l

tensor([[[[-4.4409e-16,  1.9429e-16,  5.5511e-16, -6.1062e-16,  0.0000e+00,  8.8818e-16,  0.0000e+00,  4.4409e-16],
          [-4.4409e-16, -1.7764e-15, -1.7764e-15,  3.5527e-15, -8.8818e-16,  3.5527e-15,  3.5527e-15, -7.1054e-15],
          [ 3.1919e-16,  1.1102e-16, -2.2204e-16,  0.0000e+00, -2.2204e-16, -1.1102e-16, -4.4409e-16,  8.8818e-16],
          [-4.4409e-16,  4.4409e-16, -4.4409e-16, -4.4409e-16,  0.0000e+00,  4.4409e-16, -4.4409e-16,  0.0000e+00],
          [-1.3323e-15, -8.8818e-16,  1.7764e-15,  3.5527e-15, -3.5527e-15,  3.5527e-15, -1.0658e-14,  7.1054e-15],
          [ 4.4409e-16,  4.4409e-16, -4.4409e-16, -8.8818e-16,  8.8818e-16,  0.0000e+00,  3.5527e-15,  0.0000e+00]]]], device='cuda:0', dtype=torch.float64)

In [21]:
di_r - di_l.squeeze(-1)

tensor([[[-1.7764e-15,  0.0000e+00,  8.8818e-16,  8.8818e-16, -1.4211e-14, -7.1054e-15]]], device='cuda:0', dtype=torch.float64)

In [22]:
df_r - df_l.squeeze(-1)

tensor([[[-1.8269e-14, -2.0872e-14, -1.2490e-14, -1.0214e-14, -1.2101e-14, -7.1054e-15]]], device='cuda:0', dtype=torch.float64)